# Generación de piezas musicales polifónicas con Autoencoder-CNN

In [ ]:
!pip install pypianoroll

In [22]:
import os
import numpy as np
import pandas as pd
import random
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#import tensorflow.keras.backend as K 
#import tensorflow.keras as keras
#from tensorflow.keras.utils import plot_model
#from sklearn.model_selection import train_test_split
import pypianoroll as pr
#import matplotlib.pyplot as plt

In [2]:
ENV = 'local'
if(ENV == 'colab'):
    BASE_PATH = '/content/drive/MyDrive/uni/TFG2023/generative-music/'
    from google.colab import drive
    drive.mount('/content/drive')
    
elif(ENV == 'local'):
    BASE_PATH = './'
else:
    raise Exception()

In [91]:
DATASET = 'POP909'
DATA_PATH=os.path.join(BASE_PATH, 'data')
DATASET_PATH = os.path.join(DATA_PATH, DATASET)

STORE_PATH=os.path.join(BASE_PATH,'generated')

RANDOM_SEED=616
SEQ_LEN=32
RESOLUTION=2

random.seed(RANDOM_SEED)

INSTRUMENTS = {'lpd_5': ['Drums','Piano','Guitar','Bass','Strings'],
              'POP909': ['MELODY', 'BRIDGE', 'PIANO']}
MAIN_INSTRUMENT = {'lpd_5': 'Piano',
              'POP909': 'MELODY'}

### Clases para el preprocesamiento de los datos y la compilación del modelo

In [99]:
# MAEDatasetEncoder
class MAEDatasetEncoder:
    
    def __init__(self, path, seq_len=32, delimiter='/', blend_tracks=False, with_csv=False, k=1000):
        self.path = path
        self.seq_len = seq_len
        self.delimiter = delimiter
        self.blend_tracks = blend_tracks
        self.with_csv = with_csv
        self.k = k
        self.parsed_files = self.load()
    
    def load(self):
        
        # Get song data from data_path and parse it as a pypianoroll object
        songs = {}
        
        if(self.with_csv):
            
            # reading from csv
            files_df = pd.read_csv(os.path.join(self.path,self.with_csv))
            files = files_df['npz_local'].values.tolist()
            sample = random.sample(files, k=self.k)
            print(len(list(set(sample))))

            for file in sample:
                if (file[-3:] == 'mid'):
                    try:
                        song = pr.read(os.path.join(self.path,file))
                        songs[file[:-4]] = song
                    except:
                        print(f'Parsing file {file} failed. Skipping.')
                        continue
                elif (file[-3:] == 'npz'):
                    
                    try:
                        song = pr.load(os.path.join(self.path,file))
                        if(file[:-4] in songs.keys()):
                            print(file[:-4])
                        songs[file[:-4]] = song
                    except:
                        print(f'Parsing file {file} failed. Skipping.')
                        continue
        else:
            # reading from folder
            for path,subdir,files in os.walk(self.path):
                for file in files:
                    if (file[-3:] == 'mid'):
                        try:
                            song = pr.read(os.path.join(path,file))
                            songs[file[:-4]] = song
                        except:
                            print(f'Parsing file {file} failed. Skipping.')
                            continue
                            
        print(f'Loaded {len(songs.keys())} songs.')
        return songs
        
    def encode(self):
        encoded_files = {}
        titles, files = list(self.parsed_files.keys()), list(self.parsed_files.values())
        
        
        for i,file in enumerate(files):

            # Clean short songs
            if(file.get_max_length() < self.seq_len):
                print(f'Skipping song {titles[i]}')
                continue

            encoded_file = self.encode_song(file)
            encoded_files[titles[i]] = encoded_file
        
        print(f'Loaded {len(encoded_files.keys())} songs.')
        self.encoded_files = encoded_files
    
    def encode_song(self, song):
        
        processed_tracks = {}
        
        padded = song.set_resolution(RESOLUTION).pad_to_same()
        
        if(self.blend_tracks):
            return { 'MAIN_TRACK':padded.binarize().blend().astype(float) }
        
        for t in song.tracks:
            bt = t.binarize()
            processed_tracks[bt.name] = bt.pianoroll.astype(float)
        return processed_tracks

In [100]:
encoder = MAEDatasetEncoder(DATASET_PATH, with_csv=False)

Loaded 909 songs.


In [79]:
encoder.instruments

['Drums', 'Piano', 'Guitar', 'Bass', 'Strings']

In [96]:
encoder.encode()

Loaded 909 songs.


In [98]:
encoder.encoded_files['001']

{'MELODY': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'BRIDGE': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'PIANO': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}